# 安装依赖以及检查GPU

In [1]:
!nvidia-smi

Wed Feb 25 03:25:09 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 3. 安装LLaMA Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
!pip install -e .[torch,metrics]
!pip install bitsandbytes>=0.43.0  # 4bit量化依赖

# 通过命令行进行LoRA微调

In [ ]:
%cd /content/LLaMA-Factory

# 修正后的命令：无缩进、无行尾注释、\紧贴行尾
!llamafactory-cli train \
    --stage sft \
    --do_train \
    --model_name_or_path Qwen/Qwen2-7B-Instruct \
    --dataset alpaca_gpt4_en \
    --template qwen \
    --finetuning_type lora \
    --quantization_bit 4 \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target all \
    --output_dir /content/drive/MyDrive/LLaMA-Factory/qwen2-7b-lora \
    --overwrite_output_dir \
    --cutoff_len 1024 \
    --preprocessing_num_workers 16 \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --learning_rate 5e-5 \
    --num_train_epochs 3.0 \
    --lr_scheduler_type cosine \
    --warmup_ratio 0.03 \
    --logging_steps 10 \
    --save_steps 100 \
    --plot_loss \
    --fp16

/content/LLaMA-Factory
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")
warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.
[WARNING|2026-02-25 03:27:53] llamafactory.hparams.parser:149 >> We recommend enable `upcast_layernorm` in quantized training.
[INFO|2026-02-25 03:27:54] llamafactory.hparams.parser:459 >> Process rank: 0, world size: 1, device: cuda:0, distributed training: False, compute dtype: torch.float16
config.json: 100% 663/663 [00:00<00:00, 3.11MB/s]
[INFO|configuration_utils.py:667] 2026-02-

# 主观问题微调前后对比

In [ ]:
from llmtuner import ChatModel

# 1. 定义测试问题
test_prompts = [
    "Explain quantum computing in simple terms.",
    "Write a Python function to calculate the factorial of a number.",
    "What are the benefits of meditation?",
    "Translate the following sentence into French: 'Hello, how are you today?'",
    "Give me 3 ideas for a weekend trip in the mountains."
]

# 2. 加载【微调前】的基座模型
print("="*30 + " 微调前（基座模型）回答 " + "="*30)
base_model = ChatModel(dict(
    model_name_or_path="Qwen/Qwen2-7B-Instruct",
    template="qwen",
    quantization_bit=4  # 同样用4bit量化
))
for prompt in test_prompts:
    print(f"\n【问题】{prompt}")
    print(f"【回答】{base_model.chat(prompt)[0]}")

# 3. 加载【微调后】的模型（基座+LoRA权重）
print("\n" + "="*30 + " 微调后（LoRA）回答 " + "="*30)
lora_model = ChatModel(dict(
    model_name_or_path="Qwen/Qwen2-7B-Instruct",
    adapter_name_or_path="/content/drive/MyDrive/LLaMA-Factory/qwen2-7b-lora",
    template="qwen",
    quantization_bit=4
))
for prompt in test_prompts:
    print(f"\n【问题】{prompt}")
    print(f"【回答】{lora_model.chat(prompt)[0]}")

# 客观指标计算（RLEU/ROUGE）

In [ ]:
!pip install evaluate rouge-score nltk sacrebleu
import evaluate
import nltk
nltk.download('punkt')

# 1. 加载指标
rouge = evaluate.load('rouge')
bleu = evaluate.load('sacrebleu')

# 2. 先让微调后的模型生成所有回答（为了简化，这里假设我们有参考答案，实际可用GPT-4的回答作为参考）
# 注意：Alpaca数据集本身有'reference'列，我们可以取测试集的一部分来算指标
# 这里为了演示，我们手动构造一个简单的"预测-参考"对示例
predictions = [
    "Quantum computing uses qubits that can be 0, 1, or both at once, allowing it to solve certain problems much faster than classical computers.",
    "Here's a Python factorial function: def fact(n): return 1 if n == 0 else n * fact(n-1)"
]
references = [
    ["Quantum computing leverages quantum mechanics phenomena like superposition and entanglement to process information in ways classical computers can't."],
    ["def factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)"]
]

# 3. 计算ROUGE
rouge_results = rouge.compute(predictions=predictions, references=references)
print("ROUGE指标:", rouge_results)

# 4. 计算BLEU
bleu_results = bleu.compute(predictions=predictions, references=references)
print("BLEU指标:", bleu_results)